In [1]:
import math
import matplotlib as plt
import shapely
from shapely.geometry import LineString, MultiLineString

In [2]:
test = [2.8533979930661495, 0.8350593062280169, 1.8350735010944388]
lines = [[[(90.92466560343746, 359.8743795603514), (50.64634319592733, 342.689586349763), (141.4154383126297, 0.0), (738.6390963118174, 170.50443049520254)], [(528.746321437764, 592.3512355554849), (250.68498866126174, 548.8781394008547)]], [[(738.6390963118174, 170.50443049520254), (1041.5171545767225, 487.5379093941301), (733.3747470285161, 795.9299552580342), (528.746321437764, 592.3512355554849)]], [[(250.68498866126174, 548.8781394008547), (148.7872773281997, 925.0354834403843), (84.8542414731346, 907.106489722617)], [(84.8542414731346, 907.106489722617), (75.55369137861999, 940.1296991426498), (9.845574646664318, 921.7006750889122), (72.29789360892028, 700.0236357413232)], [(72.29789360892028, 700.0236357413232), (0.0, 679.7554941019043), (90.92466560343746, 359.8743795603514)]]]

In [80]:


class PolyClusters:
    def __init__(self, coordinates, angle, **kwargs):
        self.coordinates = coordinates
        self.angle = angle
        self.line = MultiLineString([LineString(c) for c in coordinates])
        self.boundrec =list(self.line.minimum_rotated_rectangle.exterior.coords)
        self.length = self.seg_length()
        self.basesec = self.BaseSection(angle, **kwargs)
        self.sum = self.grid_options()
    #dolzhno bit opisano ranee, pomoika
    def seg_length(self):
        l = LineString([self.boundrec[0], self.boundrec[1]]).length
        return round(l)


    class BaseSection:
        def __init__(self, angle, **kwargs):
            self.angle = angle
            self.an_normalized = self.angle_normalized()
            self.types = kwargs
            self.width_dep = self.width_dependency()

        def angle_normalized(self):
            if self.angle > math.pi/4:
                if self.angle < math.pi/2:
                    ang = (math.pi/2) - self.angle
                elif 3 * math.pi/4 > self.angle > math.pi/2:
                    ang = self.angle - (math.pi/2)
                else:
                    ang = math.pi - self.angle
            else:
                ang = self.angle
            return ang

        def width_dependency (self):
            width = {}
            for key, value in self.types.items():
                new_w = value[0] + (((value[1]- value[0]) * self.an_normalized) / 0.872665) #50 degrees
                width[key]=round(new_w)
            return width


    def grid_options (self):

        def combination_sum():
            ans = []
            temp = []
            arr = sorted(list(set(self.basesec.width_dep.values())))
            arr_vals = list(self.basesec.width_dep.values())
            arr_keys = list(self.basesec.width_dep.keys())
            ln = self.length
            find_numbers(ans, arr, temp, ln, 0, arr_vals, arr_keys)
            return ans

        def find_numbers(ans, arr, temp, ln, index, arr_vals, arr_keys):
            if arr[0]/3>ln>=0:
                ans.append(sum(list(temp)) / 31)
            for i in range(index, len(arr)):
                if(ln - arr[i]) >= 0:
                    temp.append(int(arr_keys[arr_vals.index(arr[i])])*4.6)
                    find_numbers(ans, arr, temp, ln-arr[i], i, arr_vals, arr_keys)
                    temp.remove(int(arr_keys[arr_vals.index(arr[i])])*4.6)
        return combination_sum()


In [81]:
input_types = {'9':[54,74], '5':[44,56], '12':[62,89]}

ii = PolyClusters(lines[0], test[0], **input_types)
leng = ii.length
iii = ii.basesec
vals = iii.width_dep.values()
dorg = ii.sum
print(vals)
dorg

dict_values([61, 48, 71])


[10.535483870967742,
 10.683870967741935,
 11.870967741935484,
 12.316129032258063,
 12.019354838709676,
 11.72258064516129,
 12.167741935483871,
 12.316129032258063,
 13.206451612903225,
 13.651612903225804,
 13.35483870967742,
 13.799999999999999,
 13.503225806451614,
 13.206451612903225,
 13.651612903225805,
 14.541935483870967,
 14.987096774193546,
 13.354838709677418,
 13.799999999999999,
 14.690322580645159,
 15.13548387096774]